In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from udpclust.UDP_modules import dp_clustering as fdp
from udpclust import UDPClust as dp

    DATA/traj-d.dat 6
    DATA/traj-t.dat 8
    DATA/traj-4.dat 10
    DATA/traj-5.dat 10

In [3]:
fname="../test/DATA/traj-4.dat"
dim=10

In [4]:
traj=[]
for line in open(fname,'r'):
    traj.append([float(x) for x in line.split()])
traj=np.array(traj)
                                                                                                                                                                                   
print('shape of input array =',traj.shape)

shape of input array = (5096, 23)


In [5]:
maxknn=496
from scipy.spatial import distance
from scipy.spatial import cKDTree

tree=cKDTree(traj)
dmat,Nlist=tree.query(traj,k=maxknn+1,n_jobs=-1)

Nlist=Nlist[:,1:]
dmat=dmat[:,1:]
Nlist+=1

In [6]:
dmat.shape

(5096, 496)

In [7]:
Nlist

array([[ 307, 4746,   53, ..., 2249, 2746,  830],
       [4286, 3644, 2587, ..., 2730, 1200,  853],
       [ 663,  305,  680, ..., 1195, 4954, 1694],
       ..., 
       [3656, 4546, 4446, ..., 2273, 2913, 2207],
       [5081, 5077, 4224, ..., 2608, 2566, 2372],
       [4538, 4376, 4852, ..., 4027, 4943, 2125]])

In [8]:
print('dmat computed')
Npoints=traj.shape[0]

dmat computed


In [9]:
rho=np.zeros(Npoints)
rho_err=np.zeros(Npoints)
#   filter value for each frame in trj_sub (either 0 or 1) 
filt=np.zeros(Npoints,dtype=np.int32)
#   error flag                                                                                                                     
id_err=np.array(0,dtype=np.int32)
#   dimension                                                                                                     
dim=np.array(dim,dtype=np.int32)
#   Neighbour list within dc                                                                                                     
#Nlist=np.ones((Npoints,maxknn),dtype=np.int32,order='F')*-1                                                                                         
Nlist=np.array(Nlist,dtype=np.int32,order='F')
#   N. of NN taken for comp dens                                                                                                              
Nstar=np.zeros(Npoints,dtype=np.int32)

In [15]:
# 2) call fortran subroutine
import time
print('fortran locknn')
t0=time.time()
fdp.get_densities\
    (id_err,dmat,dim,rho,rho_err,filt,Nlist,Nstar)
print('Done!'),
print(time.time()-t0)

fortran locknn
Done!
0.06341671943664551


In [11]:
Nstar=np.zeros(Npoints,dtype=np.int32)

In [18]:
t0=time.time()
prefactor=fdp.prefactor(dim)
print(time.time()-t0)
Vols=prefactor*dmat**dim
print(time.time()-t0)
fdp.get_k(id_err,Vols,dim,Nlist,Nstar)
for i in range(Npoints):
    fdp.free_energy(Nstar[i],Vols[i])
print('Done!'),
print(time.time()-t0)

0.00010085105895996094
0.2965428829193115
Done!
0.7244622707366943


In [13]:
fdp.free_energy(Nstar[0],Vols[0])

8.213429824789426

In [19]:
t0=time.time()
fdp.get_dens(id_err,dmat,dim,rho,rho_err,filt,Nlist,Nstar)
print('Done!'),
print(time.time()-t0)

Done!
0.4372575283050537
